In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

## Non-closed-form solution
The non-closed-form solution for probability of default (PD) in the Merton model is given by:

$$ E = V \Phi(d_1) - D e^{-rT}\Phi(d_2) $$
$$ \sigma_E = \frac{V}{E} \Phi(d_1)\sigma_V $$

where:
- $\Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution
- $ d_1 = \frac{\ln\frac{V}{D} + (r + 0.5{\sigma_V}^2)T}{\sigma_V\sqrt{T}} $
- $ d_2 = d_1 - \sigma_V\sqrt{T} $
- $ E $ is the equity value
- $ \sigma_E $ is the equity volatility 
- $ V $ is the asset value. This is unknown and solved for.
- $ \sigma_V $ is the asset volatility. This is unknown and solved for.
- $ D $ is the debt face value.
- $ r $ is the risk-free rate.
- $ T $ is the time to maturity.

These two functions are used to solve for $ V $ and $ \sigma_V $. Once that is done, the following equation is used to find probability of default:

$$ PD = \Phi\left( \frac{\ln\left(\frac{D}{V}\right) - \left( r - \frac{1}{2} \sigma_V^2 \right) T}{\sigma_V \sqrt{T}} \right)$$



where:

- $\Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution  
- $ V $ is the value of the firm's assets  
- $ D $ is the face value of debt 
- $ r $ is the risk-free interest rate  
- $ \sigma_V $ is the volatility of the firm's assets  
- $ T $ is the time to maturity




In [3]:
def compute_merton_pd(E, sigma_E, D, r, T=1.0):
    # Bad input
    if E <= 0 or r < 0:
        return np.nan
    
    # Shortcut: if leverage is very low and sigma_E is not high, assume PD ≈ 0 to avoid solver convergence issues
    if (D == 0) or (sigma_E == 0) or ((D / E < 0.2) and (sigma_E < 0.6)):
        return 0.0
    
    # Clip to aid the solver
    sigma_E = np.clip(sigma_E, 0.05, 2.0)
    
    # Sqared residuals from first two equations
    def equations(vars):
        V, sigma_V = vars
        if V <= 0 or sigma_V <= 0:
            return 1e10  # large penalty for invalid values
        try:
            d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
            d2 = d1 - sigma_V * np.sqrt(T)
            eq1 = V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E
            eq2 = (V / E) * norm.cdf(d1) * sigma_V - sigma_E
            return eq1**2 + eq2**2
        except:
            return 1e10

    V0 = max(E + D, 1e6)
    sigma_V0 = np.clip(sigma_E * 0.9, 0.05, 2.0)
    bounds = [(1e6, 1e14), (0.01, 2.0)]
    bounds = [(1e6, 1e14), (0.01, 2.2)]
    result = minimize(equations, x0=[V0, sigma_V0], bounds=bounds, method='L-BFGS-B', options={'ftol': 1e-3})

    if result.success:
        V_opt, sigma_V_opt = result.x
        d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))
        pd = norm.cdf(-d2)
        return pd
    else:
        return np.nan

In [4]:
df= pd.read_csv('../data/clean_data.csv')
df['merton_pd'] = df.apply(
    lambda row: compute_merton_pd(
        E=row['market_cap'],
        sigma_E=row['equity_volatility'],
        D=row['total_debt'],
        r=row['rf'] / 100  # Convert percentage to decimal
    ),
    axis=1
)
print(df)

              date  permno   tic              conm     PRC           atq  \
0       2006-01-03   14593  AAPL         APPLE INC   74.75  1.418100e+10   
1       2006-01-04   14593  AAPL         APPLE INC   74.97  1.418100e+10   
2       2006-01-05   14593  AAPL         APPLE INC   74.38  1.418100e+10   
3       2006-01-06   14593  AAPL         APPLE INC   76.30  1.418100e+10   
4       2006-01-09   14593  AAPL         APPLE INC   76.05  1.418100e+10   
...            ...     ...   ...               ...     ...           ...   
441045  2024-12-24   11850   XOM  EXXON MOBIL CORP  106.40  4.619160e+11   
441046  2024-12-26   11850   XOM  EXXON MOBIL CORP  106.49  4.619160e+11   
441047  2024-12-27   11850   XOM  EXXON MOBIL CORP  106.48  4.619160e+11   
441048  2024-12-30   11850   XOM  EXXON MOBIL CORP  105.76  4.619160e+11   
441049  2024-12-31   11850   XOM  EXXON MOBIL CORP  107.57  4.534750e+11   

                dlcq         dlttq     SHROUT    market_cap    total_debt  \
0       0.

In [5]:
df['pd_valid'] = df['merton_pd'].notna()
df.to_csv("merton_model_output.csv", index=False)